In [21]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
date_format = "%Y-%m-%d"

In [75]:
os.chdir(r'C:\Users\logiusti\Lorenzo\PyWorkspace\scripts\Wrapper\data\anom1')

In [71]:
crit_seqs_path=r"C:\Users\logiusti\Lorenzo\PyWorkspace\scripts\Wrapper\data\critical_dates"
anomalities_path=r'C:\Users\logiusti\Lorenzo\PyWorkspace\scripts\Wrapper\data\anom1'

In [84]:
#crit_seq = pd.to_datetime(pd.Series(pd.read_csv(crit_seqs_path+"\\"+"ESS6_SLASH_85X"+'_crit_seq_dates.csv', header=None)[0])).sort_values().reset_index(drop=True)
crit_seq = pd.Series()
anomalities = pd.read_csv("./"+"ESS6_SLASH_85X"+'/LSTM_autoencoder_distance.csv')
anomalities = anomalities.loc[anomalities['Anomaly'] == True]
anomalities['Time'] = pd.to_datetime(anomalities['Time'], format="%Y-%m-%d %H:%M:%S.%f")

#tim off 

dayoff = ((anomalities.Time.iloc[-1] - anomalities.Time.iloc[0]).days)*.1
anomalities = anomalities.loc[anomalities['Time']  >= (pd.to_datetime(anomalities.Time[0], format="%Y-%m-%d") + pd.DateOffset(days=dayoff))]

def pick_the_peak(df):
    return df.loc[df['Loss_mae'] == max(df['Loss_mae'])]


anomalities['groups'] = (anomalities.Time.diff().fillna(pd.Timedelta(seconds=0))/np.timedelta64(7, 'D'))\
                                .gt(1).cumsum().add(1).astype(str)
anomalities.set_index('groups', drop=True, inplace=True)
danger = anomalities.groupby(level=0, group_keys=False).apply(pick_the_peak)
danger['Time'] = danger['Time'].dt.date

In [85]:
(crit_seq 

SyntaxError: unexpected EOF while parsing (<ipython-input-85-68880384f18c>, line 1)

In [86]:
for idx,row in danger.iterrows():
    start = pd.to_datetime(row.Time, format="%Y-%m-%d")
    end = start + pd.DateOffset(days=4)
    print(start, end)
    if crit_seq.loc[(crit_seq >= start ) & (crit_seq <= end) ].empty:# and row['Loss_mae'] >= 1.5*row['Threshold']:
        print("Pensa positivo\n", row)

2013-12-18 00:00:00 2013-12-22 00:00:00
Pensa positivo
 Time         2013-12-18
Loss_mae        3.20254
Threshold       2.84371
Anomaly            True
Name: 1, dtype: object
